Segmentation is applied to either 4D or 3D volumes.
- Volumes are read from directories and organized in a <code>np.array</code>;
- A volume subset is analyzed in order to determine the correct <code>threshold</code>;
- The volume is segmented using the newly found <code>threshold</code>;
- Results are imported into the <code>napari</code> viewer.

In [2]:
import numpy as np                  # type: ignore
import napari                       # type: ignore     
import myfunctions as mf            # type: ignore

#### Reading 4D sequence

In [2]:
exp = mf.exp_list()[1]
first_slice = 100
last_slice = 101
OS = 'Windows'

In [3]:
sequence = mf.read_4Dsequence(exp, first_slice=first_slice, last_slice=last_slice, OS=OS, skip180=False)

threshold = mf.find_threshold(sequence, target=5700)

segmented_sequence = mf.segment4D(sequence, threshold, smallest_3Dvolume=10, save=False, exp=exp, OS=OS)


Finding threshold...
Threshold=2.00 found in 3.10 s


Segmenting and propagating labels...


Backward propagation: 100%|██████████| 60/60 [00:02<00:00, 24.64it/s]



Filtering...
Small agglomerates removed in 0.35 s


Inconsistent agglomerates removal: 100%|██████████| 51/51 [00:03<00:00, 12.80it/s]

In [4]:
viewer = napari.Viewer()

images = [viewer.add_image(sequence, name='Volume', opacity=0.4)]

labels = [viewer.add_labels(segmented_sequence, name='Labels', blending='additive', opacity=0.8)]

settings = napari.settings.get_settings()
settings.application.playback_fps = 5
viewer.dims.current_step = (0, 0)

#### Reading 3D sequence

In [3]:
exp = 'P28A_FT_H_Exp1'
OS = 'Windows'
threshold = 2
start_time = 150
end_time = 152

sequence = mf.read_4Dsequence(exp, start_time=start_time, end_time=end_time, OS=OS, skip180=False)
segmented_sequence = mf.segment4D(sequence, threshold, smallest_3Dvolume=50, save=False, exp=exp, OS=OS)


Segmenting and propagating labels...


Backward propagation: 100%|██████████| 2/2 [01:58<00:00, 59.02s/it]



Filtering...
Small agglomerates removed in 2.63 s


Inconsistent agglomerates removal: 0it [00:00, ?it/s]

In [2]:
exp = 'P28A_FT_H_Exp1'
threshold = 2

sequence1 = mf.read_3Dsequence(exp, time=150, OS='Windows')
print('Sequence 1 read')
segmented_sequence1 = mf.segment3D(sequence1, threshold)
print('Sequence 1 segmented')
sequence2 = mf.read_3Dsequence(exp, time=151, OS='Windows')
print('Sequence 2 read')
segmented_sequence2 = mf.segment3D(sequence2, threshold)
print('Sequence 2 segmented')
segmented_sequence2 = mf.propagate_labels(segmented_sequence1, segmented_sequence2, verbose=True)
print('Labels propagated') 
sequence3 = mf.read_3Dsequence(exp, time=152, OS='Windows')
print('Sequence 3 read')
segmented_sequence3 = mf.segment3D(sequence3, threshold)
print('Sequence 3 segmented')
segmented_sequence3 = mf.propagate_labels(segmented_sequence2, segmented_sequence3, verbose=True)
print('Labels propagated') 

# mf.save_segmentation_map(segmented_sequence, exp, OS='MacOS')

Sequence 1 read
Sequence 1 segmented
Sequence 2 read
Sequence 2 segmented


Propagating labels: 100%|██████████| 807/807 [00:53<00:00, 15.06it/s]


Labels propagated
Sequence 3 read
Sequence 3 segmented


Propagating labels: 100%|██████████| 803/803 [00:43<00:00, 18.40it/s]


Labels propagated


In [3]:
viewer = napari.Viewer()
_ = viewer.add_image(sequence, opacity=0.4)
_ = viewer.add_labels(segmented_sequence, blending='additive', opacity=0.8)
settings = napari.settings.get_settings()
settings.application.playback_fps = 10
viewer.dims.current_step = (0, 0)

#### 4D segmentation algorithm description

4D segmentation is a generalization of the 3D segmentation algorithm. 

<code>segment3D</code> assigns labels to agglomerates in each 2D $(x,y)$ slice separately using the <code>mask</code> function and exploits label propagation and filtering to settle consistent labels between different slices.
- <code>propagate_labels</code> function is used to assign the same labels to overlapping agglomerates in two neighboring slices. For each agglomerate in <code>previous_mask</code>, proceeding in increasing order of agglomerate size, its label is propagated to the agglomerate with largest overlap in <code>previous_mask</code> and to all other agglomerates with overlap > <code>propatation_threshold</code>. The labels of the agglomerates that were not present in <code>previous_mask</code> are redistributed in order to assume the values of the lowest integers possible. The propagation is carried out in two directions (+z and -z) in order to improve consistency.
- Filtering consists in two processes:
    -  <code>remove_isolated_agglomerates</code> cycles through every slice and removes the agglomerates which are present in neither the previous slice nor in the next slice;
    - <code>remove_small_agglomerates</code> removes the agglomerates with volume smaller than <code>smallest_3Dvolume</code>.

<code>segment4D</code> exploits 3D segmentation to segment each volume in the time sequence separately and applies in 4D the steps that are applied in 3D by <code>segment3D</code> with some small differences.
- Each 3D volume is segmented using <code>segment3D</code> and labels are propagated (this time along the time axis, not the z axis) using <code>propagate_labels</code> function.
- Filtering in 4D exploits again the removal of small agglomerates (this time for agglomerates with volume smaller than <code>smallest_4Dvolume</code>) and introduces another filter by means of <code>remove_inconsistent_agglomerates</code>. This function is used to remove the agglomerates that are not appearing consecutively for at least (<code>time_steps</code>) time instants.